<a href="https://colab.research.google.com/github/dric2018/audioClassification/blob/main/GIZ_NLP_UGANDA_AUDIO_CLASSIFICATION.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!nvidia-smi

NVIDIA-SMI has failed because it couldn't communicate with the NVIDIA driver. Make sure that the latest NVIDIA driver is installed and running.



In [2]:
%%writefile init.sh
pip install git+https://github.com/eaedk/testing-zindi-package.git -q

apt-get install unzip unrar p7zip-full
python3 -m pip install patool -q
python3 -m pip install pyunpack -q
pip install -q torch
pip install -q pytorch-lightning 
pip install librosa
pip install efficientnet-pytorch
pip install torchaudio
pip install -U pandas # upgrade pandas
pip install swifter
mkdir {/content/models, data/datasets/images}

python init.py --username I_am_Zeus_AI --download #Connects the user and download the dataset from zindi
unzip -q data/raw/audio_files.zip -d data/
unzip -q data/raw/AdditionalUtterances.zip -d data/
unzip -q data/raw/nlp_keywords_29Oct2020.zip -d data/

python utils.py --data_path /content/data/datasets --csv_path /content/data/Giz-agri-keywords-data --create_train_df True --create_spectrograms True --specs_path /content/data/Giz-agri-keywords-data/datasets
python train.py --train_csv_path /content/data/final_train.csv --gpus 1 --test_batch_size 32 --train_batch_size 64 --kfold 3 --num_epochs 50 --img_size 224 --specs_images_path /content/data/datasets/images --save_models_to /content/models --seed_value 2020 --lr 0.0023182567385564073
python inference.py --test_csv_path /content/data/final_test.csv --models_path /content/models --sample_csv_path /content/data/SampleSubmission.csv --arch 'resnet34' --save_resulting_file_to /content/ --test_batch_size 16 --specs_images_path /content/data/datasets/images

Writing init.sh


In [3]:
%%writefile init.py

import os, sys, gc, glob
import argparse
import pandas as pd
import numpy as np
from zindi import user as zuser
from sklearn.model_selection import train_test_split, StratifiedKFold


parser = argparse.ArgumentParser(description='Logging phase')

parser.add_argument('--username', type=str, help='Your Zindi username')
parser.add_argument('--prefix', type=str, default='/content/drive/My Drive/Zindi/GIZ/', help="***")
parser.add_argument('--data', type=str, default='data/', help="***")
parser.add_argument('--seed', type=int, default=2020, help='randomness factor')
parser.add_argument('--download', action='store_true', help="")

def download(args):
	user = zuser.Zindian(args.username)
	user.which_challenge
	user.select_a_challenge()
	user.download_dataset(args.data)

def info(args):
	train = pd.read_csv(args.prefix + 'BaseTrain.csv')
	full = pd.read_csv(args.prefix + 'Train.csv')
	add = pd.read_csv(args.prefix + 'AddTrain.csv')

	lung_words = add.target.unique()
	eng_words = [w for w in train.target.unique() if w not in lung_words]

	dicts = {"base data": train, "add data": add, "full data": full}

	with open(args.prefix + "info.txt", "w") as f:
		for name, df in dicts.items():
			info = f"##{name}##. \nIt contains {df.target.nunique()} unique classes .\n"
			info += f"Shape: {df.shape}\n\n"
			f.write(info)
		f.close()
  

def main(parser):
	args = parser.parse_args()

	if args.download: download(args)

	if args.pp: 
		preprocessing(args)
		info(args)


if __name__ == '__main__':
	main(parser)

Writing init.py


In [4]:
%%writefile utils.py

import os
import shutil
from zipfile import ZipFile
from tqdm import tqdm 
from pyunpack import Archive
import argparse
import pandas as pd 
import numpy as np
import librosa
import matplotlib.pyplot as plt 
import swifter
from scipy import signal
import warnings
warnings.filterwarnings(action='ignore')



parser = argparse.ArgumentParser()
parser.add_argument('--data_path',  type=str, help='data source directory')
parser.add_argument('--destination_path',  type=str, help='data destination directory')
parser.add_argument('--extract_files', default=False, type=bool, help='execute extraction or not')
parser.add_argument('--kind', default='7z', type=str, help='For .7z files extraction')
parser.add_argument('--create_train_df', default=True, type=bool, help='Create a training dataframe or not')
parser.add_argument('--csv_path', type=str, help='Csv files path')
parser.add_argument('--specs_path', type=str, help='Spectrograms files path')
parser.add_argument('--create_spectrograms', default=True, type=bool, help='Create log spectrogram or not')
parser.add_argument('--sample_csv_path', type=str, help='sample submission csv file')


def extract_files(data_path:str, destination_path:str):
    files = os.listdir(data_path)
    dest = os.path.join(destination_path)

    os.makedirs(dest, exist_ok=True)

    for fn in tqdm(files):
        if fn.split('.')[-1] == "zip":
            try :
                with ZipFile(os.path.join(data_path, fn), "r") as zip_ref:
                    for file_ in tqdm(iterable=zip_ref.namelist(), total=len(zip_ref.namelist()), desc="Extrating files"):

                        # Extract each file to another directory
                        # If you want to extract to current working directory, don't specify path
                        zip_ref.extract(member=file_, path=dest)
                    print(f'[INFO] successfully extracted files from {fn}')

            except Exception as ex:
                print(f'[ERROR] {ex}')


def extract_files_v1(data_path:str, destination_path:str):
    files = os.listdir(data_path)
    dest = os.path.join(destination_path)

    os.makedirs(dest, exist_ok=True)

    for fn in tqdm(files):
        if fn.split('.')[-1] == "7z":
            try :
                print(f'[INFO] Extracting files from {fn}')

                tqdm(Archive(os.path.join(data_path, fn)).extractall(dest), desc=len(os.listdir(os.path.join(data_path, fn))))
                print(f'[INFO] successfully extracted files from {fn}')

            except Exception as ex:
                print(f'[ERROR] {ex}')



def calc_duration(file_path):
    signal, sr = librosa.load(file_path)
    return signal.shape[0] / sr
    

def label_to_int(label, class_dict):
    return class_dict[label]


def create_train_dataframe(csv_path, data_path):
    if ('Train.csv' in os.listdir(csv_path))  or ('train.csv' in os.listdir(csv_path)):
        try:
            df = pd.read_csv(os.path.join(csv_path, 'Train.csv'))
        except:
            df = pd.read_csv(os.path.join(csv_path, 'train.csv'))


        folder_list = os.listdir(data_path)
        files_list = []
        labels = []

        for folder in folder_list:
            if folder != 'audio_files':
                try:
                    keywords = os.listdir(os.path.join(data_path, folder))
                    for keyword in keywords:
                        files = os.listdir(os.path.join(data_path, folder, keyword))
                        files_list += [os.path.join(folder, keyword, fn) for fn in files ]
                        labels += [keyword for _ in range(len(os.listdir(os.path.join(data_path, folder, keyword)))) ]
                except:
                    pass
                
        

        df = df.append(pd.DataFrame({
            'fn' : files_list,
            'label' : labels
        }), ignore_index=True)


        df['fn'] = data_path +'/'+ df['fn']
        df['duration'] = df.swifter.progress_bar(enable=True, desc='computing audio durations').apply(lambda row : calc_duration(row.fn), axis=1) # use all available cpu cores
        df['label'] = df.swifter.progress_bar(enable=True, desc='Converting labels to ints').apply(lambda row : label_to_int(label=row.label, class_dict={l:idx for idx, l in enumerate(df.label.unique().tolist())}) , axis=1)# use all available cpu cores
        df.to_csv(os.path.join(csv_path, 'final_train.csv'), index=False)



def log_specgram(audio, sample_rate, window_size=20, step_size=10, eps=1e-10):

    """
    Borrowing log spec function from https://www.kaggle.com/davids1992/data-visualization-and-investigation
    """
    nperseg = int(round(window_size * sample_rate / 1e3))
    noverlap = int(round(step_size * sample_rate / 1e3))
    freqs, _, spec = signal.spectrogram(audio,
                                    fs=sample_rate,
                                    window='hann',
                                    nperseg=nperseg,
                                    noverlap=noverlap,
                                    detrend=False)
    return freqs, np.log(spec.T.astype(np.float32) + eps)


def wav2img(wav_path, targetdir='', figsize=(4,4)):
    """
    takes in wave file path
    and the fig size. Default 4,4 will make images 288 x 288
    """
    fig = plt.figure(figsize=figsize)    
    # use soundfile library to read in the wave files
    sound, samplerate  = librosa.load(wav_path)
    _, spectrogram = log_specgram(sound, samplerate)
    
    ## create output path
    output_file = wav_path.split('/')[-1].split('.wav')[0]
    output_file = targetdir +'/'+ output_file
    #plt.imshow(spectrogram.T, aspect='auto', origin='lower')
    plt.imsave('%s.png' % output_file, spectrogram)
    plt.close()

    return output_file+'.png'




if __name__ == '__main__':

    args = parser.parse_args()

    if args.extract_files:
        if args.kind == 'zip':
            try:
                extract_files(args.data_path, args.destination_path)
            except Exception as ex:
                raise ex
        else:
            try:
                extract_files_v1(args.data_path, args.destination_path)
            except Exception as ex:
                raise ex

    if args.create_train_df:
        try:
            df = create_train_dataframe(args.csv_path, args.data_path)
        except Exception as ex:
            print(ex)

    if args.create_spectrograms:
        try:
            # train spectrograms
            img_dir = args.specs_path+'/images'
            df['spec_path'] = img_dir
            os.makedirs(img_dir, exist_ok=True)
            for row in tqdm(df.iterrows(), total=len(df), desc='Creating specs'):
                output_file = wav2img(wav_path=row[1].fn, targetdir=img_dir)
                df.at[row[0], 'spec_path'] = output_file

            # save dataframe with specs paths
            df.to_csv(os.path.join(args.csv_path, 'final_train.csv'), index=False)
            
            # test spectrograms
            sample = pd.read_csv(args.sample_csv_path)
            sample['fn'] = args.data_path +'/'+ sample['fn']
            sample['spec_path'] = img_dir

            for row in tqdm(sample.iterrows(), total=len(sample), desc='Creating specs'):
                output_file = wav2img(wav_path=row[1].fn, targetdir=img_dir)
                sample.at[row[0], 'spec_path'] = output_file

            sample.to_csv(os.path.join(args.csv_path, 'final_test.csv'), index=False)
            
        except:
            pass

Writing utils.py


In [5]:
%%writefile datasets.py
import torch 
from torch.utils.data import Dataset, DataLoader
from keras.utils import to_categorical
import librosa
import os
import albumentations as al
from PIL import Image
import cv2
import pandas as pd 
from pytorch_lightning import seed_everything
import numpy as np

class AudioDataset(Dataset):
    def __init__(self, images_path:str,df:pd.DataFrame, transforms=None,  task = 'train', num_classes=193, one_hot=False, **kwargs):
        super(AudioDataset, self).__init__()

        self.task = task 
        self.df = df
        try:
            self.class_dict = {label:idx for idx,label in enumerate(self.df.label.unique().tolist())}
        except:
            pass
        self.transforms = transforms
        self.one_hot = one_hot
        self.images_path = images_path
        self.num_classes = num_classes
        self.log_specs = os.listdir(self.images_path)


    def __getitem__(self, index):
        wav_path = self.df.iloc[index].fn
        file_ = wav_path.split('/')[-1].split('.wav')[0]
        file_path = self.images_path +'/'+ file_ +'.png'
        
        # load spectrogram
        img = cv2.imread(file_path, cv2.IMREAD_GRAYSCALE)        
        if self.transforms is not None:
            img = self.transforms(image=img)['image']
        
        sample = {'image' : torch.tensor(img, dtype=torch.float)}

        if self.task == 'train':
            label = self.df.iloc[index].label
            if self.one_hot:
                sample.update({
                    'label' : torch.tensor(to_categorical(label, self.num_classes), dtype=torch.float)
                })            
            else:
                sample.update({
                    'label' : torch.tensor(label, dtype=torch.long)
                })
        return sample


    def __len__(self):
        return len(self.df)

    

Writing datasets.py


In [6]:
%%writefile models.py

import torch
import torch.nn as nn 
import torch.nn.functional as F
import torchvision
from torchvision import models, transforms
from torch.utils.data import Dataset, DataLoader

import numpy as np
import os
import pandas as pd 
import random
import albumentations as al

import pytorch_lightning as pl
from pytorch_lightning import Trainer, loggers, seed_everything 
from efficientnet_pytorch import EfficientNet 

from datasets import AudioDataset
import warnings
warnings.filterwarnings(action='ignore')






class AudioClassifier(pl.LightningModule):
    def __init__(self, pretrained=True, out_size=193,img_size=224, lr=0.0023182567385564073, arch_name='resnet34'):
        super(AudioClassifier, self).__init__()
        self.save_hyperparameters()

        if 'efficient' in self.hparams.arch_name:
            self.arch = Efficientnet.from_pretrained(self.hparams.arch_name)

            #change firs conv layer to accept grayscale images
            head = torch.nn.Conv2d(1, 64, kernel_size=(7,7), stride=(2,2), padding=(3,3))
            head.weight = torch.nn.Parameter(self.arch.conv1.weight.sum(dim=1, keepdim=True))
            self.conv1 = head

            # add our own  classifier 
            self.num_last_ftrs = getattr(self.arch, 'fc').in_features
            self.arch.fc = nn.Sequential(
                nn.Dropout(.5),
                nn.Linear(self.num_last_ftrs, out_size)
            ) 
            torch.nn.init.xavier_normal_(self.arch.fc[1].weight)

        else:
            self.arch = getattr(models, arch_name)(pretrained)

            head = torch.nn.Conv2d(1, 64, kernel_size=(7,7), stride=(2,2), padding=(3,3))
            head.weight = torch.nn.Parameter(self.arch.conv1.weight.sum(dim=1, keepdim=True))

            self.arch.conv1 = head
            # classifier part
            self.num_last_ftrs = getattr(self.arch, 'fc').in_features
            self.arch.fc = nn.Sequential(
                nn.Dropout(.5),
                nn.Linear(self.num_last_ftrs, out_size)
            ) 
            torch.nn.init.xavier_normal_(self.arch.fc[1].weight)



    def forward(self, x):
        x = self.arch(x.view(-1, 1, self.hparams.img_size, self.hparams.img_size))

        return x


    def configure_optimizers(self):
        opt = torch.optim.SGD(self.parameters(), lr=self.hparams.lr)
        return opt



    def training_step(self, batch, batch_idx):
        x, y = batch['image'], batch['label']
        logits = self(x)

        logLoss = self.get_loss(logits=logits, targets=y)
        acc = self.get_acc(logits=logits, targets=y)

        # logging 
        self.log('train_acc', acc, on_epoch=True, on_step=False, prog_bar=True)
        self.log('train_logLoss', logLoss, on_epoch=True, on_step=True, prog_bar=False)

        return {'loss':logLoss, 'train_logloss':logLoss, 'train_acc':acc}


    def validation_step(self, batch, batch_idx):
        x, y = batch['image'], batch['label']
        logits = self(x)

        val_loss = self.get_loss(logits=logits, targets=y)
        val_acc = self.get_acc(logits=logits, targets=y)

        # logging 
        self.log('val_acc', val_acc, on_epoch=True, on_step=False, prog_bar=True)
        self.log('val_logLoss', val_loss, on_epoch=True, on_step=False, prog_bar=True)

        return {'val_logLoss':val_loss, 'val_acc':val_acc}


    def test_step(self, batch, batch_idx):
        x, y = batch['image'], batch['label']
        logits = self(x)

        test_loss = self.get_loss(logits=logits, targets=y)
        test_acc = self.get_acc(logits=logits, targets=y)

        # logging 
        self.log('test_acc', test_acc, on_epoch=True, on_step=False, prog_bar=True)
        self.log('test_logLoss', test_loss, on_epoch=True, on_step=False, prog_bar=True)

        return {'test_logLoss':test_loss, 'test_acc':test_acc}



    def get_acc(self, logits, targets):

        preds = nn.functional.softmax(logits, dim=1).argmax(1)

        acc = (preds == targets).float().mean()
        return acc


    def get_loss(self, logits, targets):
        
        loss = nn.CrossEntropyLoss()(logits, targets)
        return loss




Writing models.py


In [7]:
%%writefile train.py
import torch
import torch.nn as nn 
import torch.nn.functional as F
import torchvision
from torchvision import models, transforms
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import KFold, StratifiedKFold, train_test_split
import numpy as np
import gc
import pandas as pd 
import random
from models import AudioClassifier
from datasets import AudioDataset
import albumentations as al
import os
import pytorch_lightning as pl
from pytorch_lightning import Trainer, loggers, seed_everything 
import argparse


# arguments parser config
parser = argparse.ArgumentParser()

parser.add_argument('--train_csv_path', type=str, help='Train csv file')
parser.add_argument('--lr', type=float, default=0.0023182567385564073,  help='Learning rate for model training')
parser.add_argument('--gpus', type=int, default=1,  help='Number of gpus to use for training')
parser.add_argument('--kfold', type=int, default=5,  help='number of folds to use for cross validation')
parser.add_argument('--train_batch_size', type=int, default=16,  help='Training batch size')
parser.add_argument('--test_batch_size', type=int, default=16,  help='Test/Evaluation batch size')
parser.add_argument('--num_epochs', type=int, default=40,  help='Number of epochs for training')
parser.add_argument('--img_size', type=int, default=224,  help='input image size')
parser.add_argument('--seed_value', type=int, default=2020,  help='Seed value for reproducibility')
parser.add_argument('--specs_images_path', type=str, help='Direcetory containing log spectrograms images')
parser.add_argument('--save_models_to', type=str, help='Directory to save trained models to')




def make_folds(data:pd.DataFrame, args, n_folds = 10, target_col='label'):
  data['fold'] = 0

  fold = StratifiedKFold(n_splits = n_folds, random_state=args.seed_value)
  for i, (tr, vr) in enumerate(fold.split(data, data[target_col])):
    data.loc[vr, 'fold'] = i

  return data, n_folds


def run_fold(fold, train_df, args,size=(224, 224), arch='resnet18', pretrained=True,   path='MODELS/', data_transforms=None):
  
  torch.cuda.empty_cache()

  fold_train = train_df[train_df.fold != fold].reset_index(drop=True)
  fold_val = train_df[train_df.fold == fold].reset_index(drop=True)

  train_ds = AudioDataset(images_path=args.specs_images_path, df=fold_train, transforms=data_transforms['train'])
  val_ds = AudioDataset(images_path=args.specs_images_path, df=fold_val, transforms=data_transforms['train'])

  trainloader = DataLoader(train_ds, batch_size=args.train_batch_size, shuffle=True , num_workers=os.cpu_count())
  validloader = DataLoader(val_ds, batch_size=args.test_batch_size, shuffle=False , num_workers=os.cpu_count())

  del train_ds
  del val_ds
  del fold_train
  del fold_val

  model = AudioClassifier(arch_name=arch, lr=args.lr, pretrained=pretrained)

  tb_logger = loggers.TensorBoardLogger(save_dir='./runs', name='ZINDI-GIZ-NLP-AGRI-KEYWORDS', version=fold)

  ckpt_callback = pl.callbacks.ModelCheckpoint(filename=f'ZINDI-GIZ-NLP-AGRI-KEYWORDS-{model.hparams.arch_name}-{fold}-based', 
                                               dirpath=path, 
                                               monitor='val_logLoss', 
                                               mode='min')
  
  trainer = Trainer(max_epochs=args.num_epochs, gpus=args.gpus, logger=tb_logger, callbacks=[ckpt_callback])

  trainer.fit(model, trainloader, validloader)


  gc.collect() # collect garbage

  return trainer.logged_metrics



if __name__=='__main__':

  args = parser.parse_args()

  _ = seed_everything(args.seed_value)
  # data augmentations
  data_transforms = {
      'train': al.Compose([
              al.Resize(args.img_size, args.img_size),
              al.Cutout(p=.6, max_h_size=15, max_w_size=10, num_holes=4),
              al.Rotate(limit=35, p=.04),
              al.Normalize((0.1307,), (0.3081,))
      ]),

      'test': al.Compose([
              al.Resize(args.img_size, args.img_size),
              al.Cutout(p=.6, max_h_size=15, max_w_size=10, num_holes=4),
              al.Normalize((0.1307,), (0.3081,))
      ])
  }

  df = pd.read_csv(args.train_csv_path)
  train, n_folds = make_folds(n_folds=args.kfold, args=args, data=df)
  
  # traiining loop
  best_fold = 0
  avg_log_loss = 0.0
  best_logloss = np.inf

  for fold in range(n_folds):

    print('')
    print('*'*18)
    print(f'Training on fold {fold}')
    print('*'*18)
    metrics = run_fold(fold=fold, train_df=train, args=args ,size=(224, 224), arch='resnet34', pretrained=True,   path=args.save_models_to, data_transforms=data_transforms)
    
    print(metrics)
    break
    print('')
    print('*'*75)
    print(f'\t\t Results for Fold {fold}')
    print('-'*75)

    print(f'> Train Acc : \t{train_acc} \t| Valid Acc : {val_acc}')
    print(f'> Train logloss : {train_loss} \t| Valid logloss : {val_loss}')
    print('-'*75)
    print(f'\t\t Results for Fold {fold}')
    print('*'*75)
    if metrics['val_logLoss'] < best_logloss:
        best_logloss = metrics['val_logLoss']
        best_fold = fold
        avg_log_loss += metrics['val_logLoss']
    else:
        avg_log_loss += metrics['val_logLoss']

  print(f'[INFO] raining done ! Avg LogLoss : {avg_log_loss / n_folds}')


Writing train.py


In [8]:
%%writefile inference.py


import torch
import torch.nn as nn 
import torch.nn.functional as F
import torchvision
from torchvision import models, transforms
from torch.utils.data import Dataset, DataLoader

import numpy as np
import os
import pandas as pd 
import random
import albumentations as al

import pytorch_lightning as pl
from pytorch_lightning import Trainer, loggers, seed_everything 
from efficientnet_pytorch import EfficientNet 

from datasets import AudioDataset
from models import AudioClassifier

import warnings
warnings.filterwarnings(action='ignore')
import argparse

from tqdm import tqdm






# arguments parser config
parser = argparse.ArgumentParser()

parser.add_argument('--test_csv_path', type=str, help='test csv file')
parser.add_argument('--sample_csv_path', type=str, help='sample submission csv file')
parser.add_argument('--gpus', type=int, default=1,  help='Number of gpus to use for inference')
parser.add_argument('--train_batch_size', type=int, default=64,  help='batch size used for training')
parser.add_argument('--test_batch_size', type=int, default=16,  help='Test/Evaluation batch size')
parser.add_argument('--n_tta', type=int, default=3,  help='Number of Test time Augmentations (TTA)')
parser.add_argument('--kfold', type=int, default=3,  help='Number of folds used for training')
parser.add_argument('--img_size', type=int, default=224,  help='input image size')
parser.add_argument('--seed_value', type=int, default=2020,  help='Seed value for reproducibility')
parser.add_argument('--specs_images_path', type=str, help='Direcetory containing log spectrograms images')
parser.add_argument('--save_resulting_file_to', type=str, help='Directory to save predictions file')
parser.add_argument('--arch', type=str, help='Model architecture to load for inference')
parser.add_argument('--num_epochs', type=int, default=40,  help='Number of epochs for training')
parser.add_argument('--models_path', type=str, help='Direcetory containing models checkpoints')
parser.add_argument('--lr', type=float, default=0.013182567385564073,  help='Learning rate for model training')




def load_models(models_path, arch=None, n_folds=3, device='cuda'):

    models = []
    for i in range(n_folds):
        models.append( AudioClassifier(arch_name=arch) )
        models[i].to(device)
        try:
            models[i].load_from_checkpoint(os.path.join(models_path, f'ZINDI-GIZ-NLP-AGRI-KEYWORDS-{arch}-{i}-based.ckpt'))
        except:
            models[i].load_from_checkpoint(os.path.join(models_path, f'ZINDI-GIZ-NLP-AGRI-KEYWORDS-{arch}-{i}-based-v0.ckpt'))
        models[i].eval()

    return models



def predict(test_df, images_path, batch_size=16, n_folds=3, transforms=None, n_tta=3, device='cuda', models=None):
    # create test AudioDataset
    test_ds = AudioDataset(images_path=images_path, task='test', df=test_df, transforms=transforms)
    test_dl = DataLoader(dataset=test_ds, shuffle=False, batch_size=batch_size)

    predictions_labels = []
    predictions_proba = []

    out = None

    for data in tqdm(test_dl):
        x = data['image'].to(device)

        for i in range(n_folds):
            if i == 0: out = models[i](x)
            else: out += models[i](x)

        out /= n_folds
        out = F.softmax(input=out, dim=1)
        out_labels = out.argmax(1)
        out_probas = out.detach().cpu().numpy()

        
        predictions_labels += out_labels.tolist()
        predictions_proba += out_probas.tolist()

    return predictions_labels ,predictions_proba



def make_submission_file(sub:pd.DataFrame,predictions_proba=None, submissions_folder=None, params=None):
    submission = pd.DataFrame()
    words = sub.columns[2:]
    submission['fn'] = sub['fn']
    for i, label in enumerate(words):
        submission[label] = 0.
    for i, label in enumerate(words):
        submission.loc[:,label] = np.array(predictions_proba)[:,i]

    train_batch_size,_, n_folds, img_size, n_epochs, arch = params.values()

    csv_file = f'GIZ_SIZE_{img_size}_arch_{arch}_n_folds_{n_folds}_num_epochs_{n_epochs}_train_bs_{train_batch_size}.csv'
    submission.to_csv(os.path.join(submissions_folder, csv_file), index=False)

    print(f'[INFO] Submission file save to {os.path.join(submissions_folder, csv_file)}')

if __name__ == '__main__':
    args = parser.parse_args()

    _ = seed_everything(args.seed_value)
    # data augmentations
    data_transforms = {
        'train': al.Compose([
                al.Resize(args.img_size, args.img_size),
                al.Cutout(p=.6, max_h_size=15, max_w_size=10, num_holes=4),
                al.Rotate(limit=35, p=.04),
                al.Normalize((0.1307,), (0.3081,))
        ]),

        'test': al.Compose([
                al.Resize(args.img_size, args.img_size),
                al.Cutout(p=.6, max_h_size=15, max_w_size=10, num_holes=4),
                al.Normalize((0.1307,), (0.3081,))
        ])
    }

    test = pd.read_csv(args.test_csv_path)
    sample = pd.read_csv(args.sample_csv_path)

    # load models
    models = load_models(models_path=args.models_path, n_folds=args.kfold, arch=args.arch)
    # make predictions
    predictions_labels, predictions_proba = predict(test_df=test, 
                                                    images_path=args.specs_images_path,
                                                    batch_size=args.test_batch_size, 
                                                    n_folds=args.kfold, 
                                                    transforms=data_transforms['test'], 
                                                    n_tta=args.n_tta, 
                                                    device='cuda', 
                                                    models=models)

    params = {
        'train_batch_size': args.train_batch_size,
        'test_batch_size':args.test_batch_size,
        'kfold': args.kfold, 
        'img_size': args.img_size,
        'epochs': args.num_epochs,
        'arch' : args.arch
    }

    make_submission_file(sub=sample,predictions_proba=predictions_proba, submissions_folder=args.save_resulting_file_to, params=params)

Writing inference.py


In [ ]:
data_dir = 